In [3]:
#THIS IS THE FINAL SCRIPT


import DataStructures
@time begin

mutable struct Info
    source::Int64
    sink::Int64
    outgoingGraph::DataStructures.DefaultOrderedDict
    incomingGraph::DataStructures.DefaultOrderedDict
    weight::DataStructures.DefaultOrderedDict
end

mutable struct DAG
    p::Info
    oGraph
    iGraph
    orderedList
end

function acyclicGraph(d::DAG)  
    #println("inside")
    if d.p.source in keys(d.iGraph)
        #println("in")
        delete!(d.iGraph, d.p.source)
        #println("deleted")
        d.iGraph[d.p.source] = []
    end
    for (k,v) in d.oGraph
        for val in v
            if val == d.p.source
                deleteat!(v, findall(x->x==d.p.source,v))
            end
        end
    end
end

function topologicalOrdering(d::DAG)
    d.orderedList = []
    candidates = [k for k in keys(d.iGraph) if length(d.iGraph[k]) == 0]
    while length(candidates) > 0
        a = pop!(candidates)
        tempList = d.p.outgoingGraph[a]
        push!(d.orderedList , a)
    
        for b in tempList
            if a in d.iGraph[b]
                deleteat!(d.iGraph[b], findall(x->x==a,d.iGraph[b]))
                #println("deleted successfully")
            else
                continue
            end
            if length(d.iGraph[b]) == 0
                push!(candidates, b)
            end
        end
    end
    return d.orderedList
end

function longestPath(d::DAG)    
    #INSIDE longestPATH
    weightDict = DataStructures.DefaultOrderedDict(0)
    calcWeight = DataStructures.DefaultOrderedDict(0)
    for i in keys(d.oGraph)
        weightDict[i] = -100000
        calcWeight[i] = -100000
    end
    weightDict[d.p.source] = 0
    calcWeight[d.p.source] = 0
    longestPath = []
    backtrackCheckList = []
    for nodes in d.orderedList
        temp = d.p.outgoingGraph[nodes]
        #println(temp)
        for node in temp
            #println(typeof(node))
            calcWeight[node] =  maximum([weightDict[nodes] + d.p.weight[string(nodes) * "->" * string(node)]])
            if calcWeight[node] > weightDict[node]
                weightDict[node] = calcWeight[node]
                push!(backtrackCheckList, nodes)
            end
        end
    end
    push!(longestPath, d.p.sink)
        while longestPath[end] != d.p.source
        prenode= pop!(backtrackCheckList)
            if string(prenode) * "->" * string(longestPath[end]) ∉ keys(d.p.weight)
            continue
        else
                if weightDict[longestPath[end]] - weightDict[prenode] == d.p.weight[string(prenode) * "->" * string(longestPath[end])]
                push!(longestPath, prenode)
            else
                continue
            end
        end
    end
    longestPath = reverse(longestPath)
    return weightDict[d.p.sink], join(longestPath, "->")   
end 
function main()
    input = []  
    open("rosalind_ba5d_762_10_dataset.txt") do f
    # read till end of file
      while ! eof(f) 
         # read a new / next line for every iteration          
         s = readline(f) 
         push!(input, s)                   
      end
    end
    outgoingGraph = DataStructures.DefaultOrderedDict(0)
    incomingGraph = DataStructures.DefaultOrderedDict(0)
    weight = DataStructures.DefaultOrderedDict(0)   
    source = parse(Int64,input[1])
    sink = parse(Int64, input[2])
    trial = input[3]
    a = DataStructures.DefaultOrderedDict(0)
    t = split(trial, ":")
    a[String(t[1])] = parse(Int64, t[2])
    path = split(t[1], "->") 

    for i in 3:length(input)
        temp = split(input[i], ":")
        weight[String(temp[1])] = parse(Int64, temp[2])
        path = split(temp[1], "->")
        if parse(Int64, path[1]) ∉ keys(outgoingGraph)
            outgoingGraph[parse(Int64, path[1])] = [parse(Int64, path[2])]
        else
            push!(outgoingGraph[parse(Int64, path[1])], parse(Int64, path[2]))
        end
        copyOutgoingGraph = deepcopy(outgoingGraph)
        for v in values(copyOutgoingGraph)
            for val in v
                if val ∉ keys(copyOutgoingGraph)
                    outgoingGraph[val] = []
                end
            end
        end
        if parse(Int64, path[2]) ∉ keys(incomingGraph)
            incomingGraph[parse(Int64, path[2])] = [parse(Int64, path[1])]
        else
            push!(incomingGraph[parse(Int64, path[2])], parse(Int64, path[1]))
        end
        copyIncomingGraph = deepcopy(incomingGraph)
        for (k,v) in copyIncomingGraph
            for val in v
                if val ∉ keys(copyIncomingGraph)
                    incomingGraph[val] = []
                end
            end
        end
    end
    
    p = Info(source, sink, outgoingGraph, incomingGraph, weight)
    oGraph = deepcopy(p.outgoingGraph)
    iGraph = deepcopy(p.incomingGraph)
    orderedList = []
    d = DAG(p, oGraph, iGraph, orderedList)
    acyclicGraph(d) 
    topologicalOrdering(d)
    x, y = longestPath(d)
    println(x)
    println(y)    
end    
main()
end

185
1->8->11->12->13->14->17->19
  0.914238 seconds (1.21 M allocations: 61.481 MiB, 3.60% gc time, 98.90% compilation time)
